##### Agent sử dụng tool có sẵn tích hợp 3rd party

In [2]:
from configparser import ConfigParser
from langchain.llms import OpenAI
import os

config = ConfigParser()
config.read("../config.ini")
os.environ["OPENAI_API_KEY"]=config["KEY"]["openai_key"]
os.environ["TAVILY_API_KEY"]  = config["KEY"]["tavily_key"]


## Create Tools

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [11]:
search = TavilySearchResults()

In [12]:
search

TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))

In [13]:
result = search.invoke(
        {
                "query": "what is quantum machnics",
                "include_answer":False
        }
)

In [14]:
result

[{'url': 'https://brilliant.org/wiki/quantum-mechanics/',
  'content': "Quantum Mechanics. Quantum mechanics [QM] is a branch of physics which describes physical systems so that properties like the energy or angular momentum are discrete quantities that are multiples of a smallest unit or quantum. A famous physicist named Erwin Schrödinger made an example of quantum mechanics, often called Schrödinger's cat."},
 {'url': 'https://scitechdaily.com/science-simplified-what-is-quantum-mechanics/',
  'content': 'Quantum mechanics is a theory that deals with the most fundamental bits of matter, energy and light and the ways they interact with each other to make up the world. This landmark theory originated in the early 20th century and is finding many real-world applications in the 21st century. Applying quantum mechanics in the laboratory, Argonne ...'},
 {'url': 'https://www.livescience.com/33816-quantum-mechanics-explanation.html',
  'content': "How is quantum mechanics different from clas

In [15]:
tools = [search]

## Create LLM Model

In [16]:
from langchain_openai import ChatOpenAI

In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.2)

## Create Prompt

In [18]:
from langchain import hub

In [19]:
prompt = hub.pull("hwchase17/openai-functions-agent")

/Users/chessman/miniforge3/envs/PyJupyter/lib/python3.10/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [20]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

## Create Chain Agent

In [22]:
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

## Create Agent Executor

In [24]:
agent_executor = AgentExecutor(agent=agent, tools=tools, prompt=prompt, verbose=True)

In [25]:
agent_executor.invoke({
        "input":"what is the weather in Ho Chi Minh City, Viet Nam"
})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Ho Chi Minh City, Viet Nam'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Ho Chi Minh City', 'region': '', 'country': 'Vietnam', 'lat': 10.75, 'lon': 106.6667, 'tz_id': 'Asia/Ho_Chi_Minh', 'localtime_epoch': 1727976194, 'localtime': '2024-10-04 00:23'}, 'current': {'last_updated_epoch': 1727975700, 'last_updated': '2024-10-04 00:15', 'temp_c': 26.3, 'temp_f': 79.3, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 243, 'wind_dir': 'WSW', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.13, 'precip_in': 0.01, 'humidity': 94, 'cloud': 0, 'feelslike_c': 30.5, 'feelslike_f': 86.9, 'windchill_c': 25.0, 'windchill_f': 77.0, 'heatindex_c': 28.0, 'heatindex_f': 82.4, 'dewpoint_c': 23.8, 'dewpoint_f': 74.8, 'vis_km': 6.0, 'vis_mi

{'input': 'what is the weather in Ho Chi Minh City, Viet Nam',
 'output': 'The current weather in Ho Chi Minh City, Vietnam is as follows:\n- Temperature: 26.3°C (79.3°F)\n- Condition: Clear\n- Wind: 3.6 km/h (WSW)\n- Humidity: 94%\n- Precipitation: 0.01 in\n\nIf you need more detailed information or a forecast, feel free to ask!'}

## Create Memory

In [27]:
from langchain.schema import AIMessage, HumanMessage

### Manual

In [28]:
agent_executor.invoke(
        {
                "chat_history":[
                        HumanMessage(content="Hello, my name is Eric"),
                        AIMessage(content="Hi Eric, How can I help you")
                ],
                "input":"what is my name"
        }
)



> Entering new AgentExecutor chain...
Your name is Eric.

> Finished chain.


{'chat_history': [HumanMessage(content='Hello, my name is Eric', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Eric, How can I help you', additional_kwargs={}, response_metadata={})],
 'input': 'what is my name',
 'output': 'Your name is Eric.'}

### Auto

In [41]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [42]:
message_history = ChatMessageHistory()

In [43]:
agent_with_message_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: message_history,
        input_messages_key="input",
        history_messages_key="chat_history"
)

In [45]:
agent_with_message_history.invoke(
        {"input":"Hi, my name is Red"},
        config={"configurable":{"session_id": "<foo>"}}
)



> Entering new AgentExecutor chain...
Hello Red! How can I assist you today?

> Finished chain.


{'input': 'Hi, my name is Red',
 'chat_history': [],
 'output': 'Hello Red! How can I assist you today?'}

In [46]:
agent_with_message_history.invoke(
        {"input":"what is my name"},
        config={"configurable":{"session_id": "<foo>"}}
)



> Entering new AgentExecutor chain...
Your name is Red. How can I help you further, Red?

> Finished chain.


{'input': 'what is my name',
 'chat_history': [HumanMessage(content='Hi, my name is Red', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Red! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Red. How can I help you further, Red?'}